In [ ]:
from lore_sa.dataset import TabularDataset

In [ ]:
target = 'income'

In [ ]:
dataset = TabularDataset.from_csv('adult.csv', class_name = target)
dataset.df.dropna(inplace = True)

In [ ]:
dataset.df.keys()

In [ ]:
dataset.df.drop(["marital-status", "fnlwgt", "educational-num", "occupation", "native-country"], axis=1, inplace=True)

In [ ]:
dataset.update_descriptor()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox

def train_model(dataset: TabularDataset):
    numeric_indices = [v['index'] for v in dataset.descriptor['numeric'].values()]
    categorical_indices = [v['index'] for v in dataset.descriptor['categorical'].values()]
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_indices),
            ('cat', OrdinalEncoder(), categorical_indices)
        ]
    )
    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    X = dataset.df.drop([target], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X.values, dataset.df[target].values,
                test_size=0.3, random_state=42, stratify=dataset.df[target].values)
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
bbox = train_model(dataset)

In [ ]:
from lore_sa.lore import TabularGeneticGeneratorLore

tabularLore = TabularGeneticGeneratorLore(bbox, dataset)

In [ ]:
out = []
for k in dataset.descriptor.keys():
    if k != 'target':
        out.extend(list(dataset.descriptor[k].keys()))
out, dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [12]:
from lore_sa.webapp import Webapp
webapp = Webapp()

In [13]:
# x = (dataset.df.drop([target], axis=1)).iloc[1]
# tabularLore.explain(x)

In [14]:
# webapp.launch_demo()
webapp.interactive_explanation(tabularLore.bbox, tabularLore.dataset, target)

INFO:     Started server process [4940]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Launching LORE_sa explanation viz webapp
Starting API server on 0.0.0.0:8000
INFO:     127.0.0.1:55954 - "GET /api/get-datasets HTTP/1.1" 200 OK
API server is ready at http://localhost:8000
NPM version detected: 10.9.3
Dependencies already installed, skipping npm install
Application started successfully!
API: http://localhost:8000
Client: http://localhost:8080


INFO:     127.0.0.1:55965 - "GET /api/check-custom-data HTTP/1.1" 200 OK
INFO:     127.0.0.1:55965 - "OPTIONS /api/explain HTTP/1.1" 200 OK
INFO:     127.0.0.1:55965 - "POST /api/explain HTTP/1.1" 200 OK
INFO:     127.0.0.1:55965 - "GET /api/get-classes-colors?method=umap HTTP/1.1" 200 OK
